# **ETL de la tabla homicidios-hechos**  
Importamos las librerias necesarias:

In [1]:
import pandas as pd

### **Extracción**
Traemos el dataset en su estado original:

In [2]:
df_homicidios = pd.read_csv("./datasets_originales/homicidios-hechos.csv", encoding="utf-8")

# Creamos una copia para trabajar comodamente:
df_homicidios_copy = df_homicidios.copy()
df_homicidios_copy.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,1/1/2016,2016,1,1,4:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,1/2/2016,2016,1,2,1:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,1/3/2016,2016,1,3,7:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


Buscamos información del dataset:

In [3]:
df_homicidios_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     696 non-null    object 
 1   N_VICTIMAS             696 non-null    int64  
 2   FECHA                  696 non-null    object 
 3   AAAA                   696 non-null    int64  
 4   MM                     696 non-null    int64  
 5   DD                     696 non-null    int64  
 6   HORA                   696 non-null    object 
 7   HH                     696 non-null    object 
 8   LUGAR_DEL_HECHO        696 non-null    object 
 9   TIPO_DE_CALLE          696 non-null    object 
 10  Calle                  695 non-null    object 
 11  Altura                 129 non-null    float64
 12  Cruce                  525 non-null    object 
 13  Dirección Normalizada  688 non-null    object 
 14  COMUNA                 696 non-null    int64  
 15  XY (CA

In [4]:
df_homicidios_copy.isna().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

**Como podemos observar las únicas columnas con datos nulos son "Calle", "Altura", "Cruce" y "Dirección Normalizada".**    
**El caso de "Altura" es que debido a su gran cantidad de valores faltantes sera eliminada. El resto sera tratada en el EDA.**

### **Transformación**  
Empezaremos por remover las columnas que son innecesarias

Las columnas innecesarias son: *'AAAA', 'MM', 'DD', 'XY (CABA)'*

In [5]:
drops = ['AAAA', 'MM', 'DD', 'XY (CABA)']
df_homicidios_copy.drop(drops, axis=1, inplace=True)

La columna "Altura" sera eliminada por su falta de información:

In [6]:
df_homicidios_copy.drop("Altura", axis=1, inplace=True)

El caso de "Cruce" es diferente, los valores faltantes parecen ser cuando el lugar del suceso no es la intersección entre dos vías, en cambio, cuando en "Dirección Normalizada" están unidas dos dirrecciones, si hay un dato.

In [7]:
df_homicidios_copy[["Calle", "Cruce", "Dirección Normalizada"]].head(5)

,Calle,Cruce,Dirección Normalizada
0,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G..."
1,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV."
2,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034
3,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL."
4,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES."


In [8]:
df_homicidios_copy[df_homicidios_copy['Cruce'].isnull()][["Calle", "Cruce", "Dirección Normalizada"]].head(5)

,Calle,Cruce,Dirección Normalizada
2,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034
9,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 1366
14,SUIPACHA,NaN,SUIPACHA 156
33,LIMA,NaN,LIMA 1483
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI


Para su mejor manipulación, cambiaremos el tipo de dato de cada columna de fechas y horas que no posea el tipo de dato correcto:

In [9]:
df_homicidios_copy['FECHA'] = pd.to_datetime(df_homicidios_copy['FECHA'], format='%m/%d/%Y')

# La columna 'FECHA' estaba en el formato mes, día, año. Ahora la cambiamos a día, mes y año:
df_homicidios_copy['FECHA'] = df_homicidios_copy['FECHA'].dt.strftime('%d/%m/%Y')

# Convertir 'HORA' a formato time
df_homicidios_copy['HORA'] = pd.to_datetime(df_homicidios_copy['HORA'], format='%H:%M:%S', errors='coerce').dt.time

In [10]:
df_homicidios_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     696 non-null    object
 1   N_VICTIMAS             696 non-null    int64 
 2   FECHA                  696 non-null    object
 3   HORA                   695 non-null    object
 4   HH                     696 non-null    object
 5   LUGAR_DEL_HECHO        696 non-null    object
 6   TIPO_DE_CALLE          696 non-null    object
 7   Calle                  695 non-null    object
 8   Cruce                  525 non-null    object
 9   Dirección Normalizada  688 non-null    object
 10  COMUNA                 696 non-null    int64 
 11  pos x                  696 non-null    object
 12  pos y                  696 non-null    object
 13  PARTICIPANTES          696 non-null    object
 14  VICTIMA                696 non-null    object
 15  ACUSADO                

**Normalizamos los nombres de las columnas**

In [11]:
df_homicidios_copy.rename(columns={
    'N_VICTIMAS': 'NUMERO_VICTIMAS',
    'HORA':"TIEMPO",
    'HH': 'HORA',
    'Calle': 'CALLE',
    'Cruce': 'CRUCE',
    'Dirección Normalizada': 'DIRECCION_NORMALIZADA',
    'pos x': 'POS_X',
    'pos y': 'POS_Y'
}, inplace=True)

# Vemos el resultado
df_homicidios_copy.head(2)

,ID,NUMERO_VICTIMAS,FECHA,TIEMPO,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,CRUCE,DIRECCION_NORMALIZADA,COMUNA,POS_X,POS_Y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,01/01/2016,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,02/01/2016,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


In [12]:
df_homicidios_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     696 non-null    object
 1   NUMERO_VICTIMAS        696 non-null    int64 
 2   FECHA                  696 non-null    object
 3   TIEMPO                 695 non-null    object
 4   HORA                   696 non-null    object
 5   LUGAR_DEL_HECHO        696 non-null    object
 6   TIPO_DE_CALLE          696 non-null    object
 7   CALLE                  695 non-null    object
 8   CRUCE                  525 non-null    object
 9   DIRECCION_NORMALIZADA  688 non-null    object
 10  COMUNA                 696 non-null    int64 
 11  POS_X                  696 non-null    object
 12  POS_Y                  696 non-null    object
 13  PARTICIPANTES          696 non-null    object
 14  VICTIMA                696 non-null    object
 15  ACUSADO                

### **Carga**  
Cargamos el dataframe en un nuevo archivo CSV en la carpeta "datasets_tranformados".

In [13]:
df_homicidios_copy.to_csv("./datasets_transformados/hechos.csv", index=False, encoding="utf-8")